# User defined methods

In some instances, you may wish to load data into the phase space which is not included in either the required quantities, or the allowed quantities which can be derived from these. This example shows how this can be achieved.

It is important to note that any additional quantites you load must still be included in the [allowed columns](https://bwheelz36.github.io/ParticlePhaseSpace/phase_space_format.html#allowed-columns), with the fill_method defined as `None`.

## Example scenario

As described [here](https://github.com/bwheelz36/ParticlePhaseSpace/issues/144) Topas is one program which can report on a lot of data which is not included in the basic phase space quantites this code handles by default. This example shows you these quantites can be read in.

The first step is to create our user defined methods to read the data. The below ugly code simply cycles through the topas data and tries to extract different quantities from it. don't worry about the format too much for now.

In [2]:
import sys
sys.path.append('../')
from ParticlePhaseSpace._ParticlePhaseSpace import _PhaseSpace_MethodHolder
from ParticlePhaseSpace import DataLoaders, PhaseSpace
from pathlib import Path
import topas2numpy as tp
import warnings

class MyMethods(_PhaseSpace_MethodHolder):

    def load_additional_topas_quantities(self, phase_space_location: (Path, str), quantities_to_read: (list, None) = None):
        """

        :param phase_space_location: location of phase space data
        :param quantities_to_read: list of quantities to read, taken from __phase_space_config__. Code will attempt
            to extract these from phase_space, and warn if it is not possible
        :return:
        """

        if isinstance(phase_space_location, str):
            phase_space_location = Path(phase_space_location)
        if not isinstance(phase_space_location, Path):
            raise TypeError(f'phase_space_location must be string or Path, not {type(phase_space_location)}')

        topas_phase_space = tp.read_ntuple(str(phase_space_location))
        fields_in_phase_space = topas_phase_space.dtype.names
        if quantities_to_read is None:
            quantities_to_read = ['first_history_flag',
                                  'time_of_flight',
                                  'run_id',
                                  'event_id',
                                  'track_id',
                                  'parent_id',
                                  'intial_kinetic_energy',
                                  'initial_cosine_x',
                                  'initial_cosine_y',
                                  'initial_cosine_z',
                                  'vertex_x',
                                  'vertex_y',
                                  'vertex_z']
        # set up dict to handle differently named quantities (can be cleaned up!)
        topas_aliases = {'first_history_flag': 'Flag to tell if this is the First Scored Particle from this History (1 means true)',
                          'time_of_flight': 'Time of Flight [ns]',
                          'run_id': 'Run ID',
                          'event_id': 'Event ID',
                          'track_id': 'Track ID',
                          'parent_id': 'Parent ID',
                          'intial_kinetic_energy': 'Initial Kinetic Energy [MeV]',
                          'initial_cosine_x': 'Initial Direction Cosine X',
                          'initial_cosine_y': 'Initial Direction Cosine Y',
                          'initial_cosine_z': 'Initial Direction Cosine Z',
                          'vertex_x': 'Vertex Position X [cm]',
                          'vertex_y': 'Vertex Position Y [cm]',
                          'vertex_z': 'Vertex Position Z [cm]'}

        for quantity in quantities_to_read:
            if quantity in topas_aliases.keys():
                _quantity = topas_aliases[quantity]
            else:
                _quantity = quantity
            if _quantity in fields_in_phase_space:
                self._PS._ps_data[quantity] = topas_phase_space[_quantity]
            else:
                warnings.warn(f'oh no. {quantity} is not in fields')

    def operate_on_additional_quantities(self):
        pass

The next step is to test this by readin in some data:

In [3]:

data_loc = Path('/home/brendan/Documents/temp/tiba_topa_testa/temp/Results/test_PS.phsp')
ps_data = DataLoaders.Load_TopasData(data_loc)
PS = PhaseSpace(ps_data, UserDefinedMethods=MyMethods)

PS.user_defined.load_additional_topas_quantities(data_loc)
PS.ps_data.columns

Index(['particle type [pdg_code]', 'x [mm]', 'y [mm]', 'z [mm]', 'weight',
       'particle id', 'time [ps]', 'px [MeV/c]', 'py [MeV/c]', 'pz [MeV/c]',
       'first_history_flag', 'time_of_flight', 'run_id', 'event_id',
       'track_id', 'parent_id', 'intial_kinetic_energy', 'initial_cosine_x',
       'initial_cosine_y', 'initial_cosine_z', 'vertex_x', 'vertex_y',
       'vertex_z'],
      dtype='object')

OK! this has worked. the quantities are in the phase space.
Note that at the moment, user defined quantities like these are not subjected to the unit system, so it is completely the user responsibility to keep track of units.